In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Korak 1: Določite Pydantic modele za strukturirane izhode

Ti modeli določajo **shemo**, ki jo bodo agenti vrnili. Uporaba `response_format` s Pydantic zagotavlja:
- ✅ Varnost tipov pri izvleku podatkov
- ✅ Samodejno preverjanje veljavnosti
- ✅ Brez napak pri razčlenjevanju prosto oblikovanih odgovorov
- ✅ Enostavno pogojno usmerjanje na podlagi polj


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Korak 2: Ustvarite orodje za rezervacijo hotela

To orodje bo klical **availability_agent** za preverjanje razpoložljivosti sob. Uporabljamo dekorater `@ai_function` za:
- Pretvorbo Python funkcije v orodje, ki ga lahko kliče AI
- Samodejno generiranje JSON sheme za LLM
- Upravljanje preverjanja parametrov
- Omogočanje samodejnega klicanja s strani agentov

Za ta prikaz:
- **Stockholm, Seattle, Tokio, London, Amsterdam** → Imajo razpoložljive sobe ✅
- **Vsa ostala mesta** → Ni sob ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Korak 3: Določanje funkcij pogojev za usmerjanje

Te funkcije pregledajo odziv agenta in določijo, katero pot slediti v delovnem toku.

**Ključni vzorec:**
1. Preverite, ali je sporočilo `AgentExecutorResponse`
2. Razčlenite strukturiran izhod (Pydantic model)
3. Vrnite `True` ali `False` za nadzor usmerjanja

Delovni tok bo ocenil te pogoje na **povezavah**, da se odloči, katerega izvršitelja poklicati naslednjega.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Korak 4: Ustvarite po meri prikazovalnika izvrševalca

Izvrševalci so sestavni deli delovnih tokov, ki izvajajo transformacije ali stranske učinke. Uporabimo dekorator `@executor`, da ustvarimo po meri izvrševalca, ki prikaže končni rezultat.

**Ključni pojmi:**
- `@executor(id="...")` - Registrira funkcijo kot izvrševalca delovnega toka
- `WorkflowContext[Never, str]` - Namigi za tipe vhodnih/izhodnih podatkov
- `ctx.yield_output(...)` - Vrne končni rezultat delovnega toka


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Korak 5: Naloži okoljske spremenljivke

Konfigurirajte LLM odjemalca. Ta primer deluje z:
- **GitHub modeli** (brezplačni nivo z GitHub žetonom)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Korak 6: Ustvarjanje AI agentov s strukturiranimi izhodi

Ustvarimo **tri specializirane agente**, od katerih je vsak ovit v `AgentExecutor`:

1. **availability_agent** - Preverja razpoložljivost hotela z uporabo orodja
2. **alternative_agent** - Predlaga alternativna mesta (ko ni sob)
3. **booking_agent** - Spodbuja rezervacijo (ko so sobe na voljo)

**Ključne značilnosti:**
- `tools=[hotel_booking]` - Agentu zagotavlja orodje
- `response_format=PydanticModel` - Zavezuje strukturiran JSON izhod
- `AgentExecutor(..., id="...")` - Ovije agenta za uporabo v poteku dela


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Korak 7: Izgradnja delovnega procesa s pogojnimi povezavami

Zdaj uporabimo `WorkflowBuilder` za ustvarjanje grafa s pogojnim usmerjanjem:

**Struktura delovnega procesa:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Ključne metode:**
- `.set_start_executor(...)` - Nastavi vstopno točko
- `.add_edge(from, to, condition=...)` - Doda pogojno povezavo
- `.build()` - Zaključi delovni proces


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Korak 8: Zaženi testni primer 1 - Mesto BREZ razpoložljivosti (Pariz)

Preizkusimo pot **brez razpoložljivosti** z zahtevkom za hotele v Parizu (ki v naši simulaciji nima sob).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Korak 9: Zaženi testni primer 2 - Mesto Z NA VOLJO (Stockholm)

Zdaj preizkusimo pot **razpoložljivosti** z zahtevkom za hotele v Stockholmu (ki ima sobe v naši simulaciji).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Glavne ugotovitve in naslednji koraki

### ✅ Kaj ste se naučili:

1. **Vzorec WorkflowBuilder**
   - Uporabite `.set_start_executor()` za določitev izhodišča
   - Uporabite `.add_edge(from, to, condition=...)` za pogojno usmerjanje
   - Pokličite `.build()` za dokončanje poteka dela

2. **Pogojno usmerjanje**
   - Funkcije pogojev pregledujejo `AgentExecutorResponse`
   - Razčlenijo strukturirane izhode za sprejemanje odločitev o poti
   - Vrnite `True` za aktiviranje povezave, `False` za preskok

3. **Integracija orodij**
   - Uporabite `@ai_function` za pretvorbo Python funkcij v AI orodja
   - Agentji samodejno kličejo orodja, ko je potrebno
   - Orodja vračajo JSON, ki ga agenti lahko razčlenijo

4. **Strukturirani izhodi**
   - Uporabite Pydantic modele za varen izvoz podatkov glede na tip
   - Nastavite `response_format=MyModel` ob ustvarjanju agentov
   - Odzive razčlenite z `Model.model_validate_json()`

5. **Lastni izvajalci**
   - Uporabite `@executor(id="...")` za ustvarjanje komponent poteka dela
   - Izvajalci lahko prenašajo podatke ali izvajajo stranske učinke
   - Uporabite `ctx.yield_output()` za oddajo rezultatov poteka dela

### 🚀 Uporaba v resničnem svetu:

- **Rezervacije potovanj**: Preverjanje razpoložljivosti, predlogi alternativ, primerjava možnosti
- **Storitve za stranke**: Usmerjanje glede na vrsto težave, ton, prednost
- **E-trgovina**: Preverjanje zalog, predlogi alternativ, obdelava naročil
- **Moderiranje vsebine**: Usmerjanje glede na ocene toksičnosti, oznake uporabnikov
- **Poteki odobritev**: Usmerjanje glede na znesek, vlogo uporabnika, stopnjo tveganja
- **Večstopenjska obdelava**: Usmerjanje glede na kakovost podatkov, popolnost

### 📚 Naslednji koraki:

- Dodajte bolj zapletene pogoje (več kriterijev)
- Implementirajte zanke z upravljanjem stanja poteka dela
- Dodajte pod-potek dela za ponovno uporabo komponent
- Integrirajte z resničnimi API-ji (rezervacije hotelov, sistemi zalog)
- Dodajte upravljanje napak in poti za izhod v sili
- Vizualizirajte poteke dela z vgrajenimi orodji za vizualizacijo


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Opozorilo**:
Ta dokument je bil preveden z uporabo storitve umetne inteligence za prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v izvorni jezik velja za avtoritativni vir. Za pomembne informacije priporočamo strokovni človeški prevod. Ne odgovarjamo za morebitna nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
